# Stochastic Unit Commitment Problem

In [51]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg

from sddip import config, storage, utils
from sddip import dualsolver, ucmodel, scenarios

## Data Processing

In [52]:
test_case_raw_dir = "WB3/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")
scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, sep="\s+")
branch_df = pd.read_csv(branch_file_raw, sep="\s+")
gen_df = pd.read_csv(gen_file_raw, sep="\s+")
gen_cost_df = pd.read_csv(gen_cost_file_raw, sep="\s+")

scenario_df = pd.read_csv(scenario_data_file, sep="\s+")

### Bus Data

In [53]:
bus_df

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1,3,0.0,0.0,0,0,1,0.95,0,0,1,1.05,0.95
1,2,1,120.0,86.0,0,0,1,0.95,-55,0,1,1.05,0.95
2,3,1,68.0,50.0,0,0,1,0.95,-59,0,1,1.05,0.95


### Branch Data

In [54]:
branch_df

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
0,1,2,0.0400,0.2000,0.000,990000,0,0,0,0,1,-360,360
1,2,3,0.0139,0.0605,2.459,990000,0,0,0,0,1,-360,360


### Generator Data

In [55]:
gen_df

,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
0,1,300,129,3000,-3000,1.1,100,1,5000,0,...,0,0,0,0,0,0,0,0,0,0


### Generator Cost Data

In [56]:
gen_cost_df

,type,startup,shutdown,n,c2,c1,c0
0,2,0,0,3,0,2,0


In [57]:
scenario_df

,t,n,p,Pd1,Pd2,Pd3
0,1,1,1.0,0,62.320833,34.305092
1,2,1,0.5,0,120.029991,68.817838
2,2,2,0.5,0,131.974918,68.153228
3,3,1,0.5,0,127.012028,65.641507
4,3,2,0.5,0,130.327922,69.694025


### Power Transfer Distribution Factor

In [58]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

zeros_col = np.zeros((1,ptdf.shape[1]))

ptdf = np.insert(ptdf, ref_bus-1, zeros_col, axis=1)

ptdf

array([[ 0.00000000e+00, -1.00000000e+00, -1.00000000e+00],
       [ 0.00000000e+00, -1.86387445e-16, -1.00000000e+00]])

### Parameter Organization

In [59]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 1000)
rg_down_max = np.full(n_gens, 1000)
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

prob = []
p_d = []

for t in range(n_stages):
    stage_df = scenario_df[scenario_df["t"] == t+1]
    p_d.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Pd')]].values.tolist())
    prob.append(stage_df["p"].values.tolist())


########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [60]:
# prob[t][n]
# Probability of realization n at stage t
prob

[[1.0], [0.5, 0.5], [0.5, 0.5]]

In [61]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
p_d

[[[0.0, 62.32083260934583, 34.305091739820405]],
 [[0.0, 120.02999105714431, 68.8178383561837],
  [0.0, 131.97491816753634, 68.15322777275095]],
 [[0.0, 127.01202751854132, 65.64150717257031],
  [0.0, 130.3279223545401, 69.6940248570091]]]

In [62]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
ex_pd

[array([ 0.        , 62.32083261, 34.30509174]),
 array([  0.        , 126.00245461,  68.48553306]),
 array([  0.        , 128.66997494,  67.66776601])]

## SDDiP

In [63]:
# Result keys
x_key = "x"
y_key = "y"
z_x_key = "zx"
z_y_key = "zy"
primal_solution_keys = [x_key, y_key, z_x_key, z_y_key]

dv_key = "dual_value"
dm_key = "dual_multiplier"
dual_solution_keys = [dv_key, dm_key]

ci_key = "intercepts"
cg_key = "gradients"
bm_key = "multipliers"
cut_coefficient_keys = [ci_key, cg_key, bm_key]


# Result storage
ps_storage = storage.ResultStorage(primal_solution_keys)
ds_storage = storage.ResultStorage(dual_solution_keys)
cc_storage = storage.ResultStorage(cut_coefficient_keys)


### Forward pass

In [64]:
penalty = 10000
cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist() + [penalty]*2
theta_lb = 0
x_trial_point = [0]*n_gens
y_trial_point = [0]*n_gens


# Sampling
n_samples = 2
samples = scenarios.ScenarioSampler(n_stages, 2).generate_samples(n_samples)

v_opt_k = []
i = 0
for k in range(n_samples):
    v_opt_k.append(0)
    for t, n in zip(range(n_stages), samples[k]):

        # Create forward model
        uc_fw = ucmodel.ForwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

        uc_fw.add_objective(cost_coeffs)

        uc_fw.add_balance_constraints(sum(p_d[t][n]))

        uc_fw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

        uc_fw.add_generator_constraints(pg_min, pg_max)

        uc_fw.add_startup_shutdown_constraints()

        uc_fw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

        uc_fw.add_copy_constraints(x_trial_point, y_trial_point)

        #TODO Lower bound
        uc_fw.add_cut_lower_bound(theta_lb)
        
        if i>0:
            cut_coefficients = cc_storage.get_stage_result(t)
            uc_fw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key])
        
        # Solve problem
        uc_fw.disable_output()
        uc_fw.model.optimize()
        model = uc_fw.model
        uc_fw.model.printAttr("X")

        # Store xtik, ytik, ztik, vtik
        x_kt = [x_g.x for x_g in uc_fw.x]
        y_kt = [y_g.x for y_g in uc_fw.y]
        z_x_kt = [z_g.x for z_g in uc_fw.z_x]
        z_y_kt = [z_g.x for z_g in uc_fw.z_y]
        s_up_kt = [s_up_g.x for s_up_g in uc_fw.s_up]
        s_down_kt = [s_down_g.x for s_down_g in uc_fw.s_down]
        
        # Value of stage t objective function
        print(f"Objective value t={t}, n={n}: {uc_fw.model.getObjective().getValue()}")
        v_opt_kt = uc_fw.model.getObjective().getValue() - uc_fw.theta.x

        v_opt_k[-1] += v_opt_kt

        # New trial point
        x_trial_point = x_kt
        y_trial_point = y_kt

        ps_dict = ps_storage.create_empty_result_dict()
        ps_dict[x_key] = x_kt
        ps_dict[y_key] = y_kt
        ps_dict[z_x_key] = z_x_kt
        ps_dict[z_y_key] = z_y_kt
        
        ps_storage.add_result(i, k, t, ps_dict)

Objective value t=0, n=0: 193.25184869833248
Objective value t=1, n=1: 400.2562918805746
Objective value t=2, n=0: 385.3070693822233
Objective value t=0, n=0: 193.25184869833248
Objective value t=1, n=0: 377.69565882665603
Objective value t=2, n=0: 385.3070693822233


#### Statistical Upper Bound

In [65]:
v_opt_k = np.array(v_opt_k)

v_mean = np.mean(v_opt_k)
v_std = np.std(v_opt_k)
alpha = 0.05

# Lower limit of the confidence interval
v_upper = v_mean + stats.norm.ppf(alpha/2)*v_std/np.sqrt(n_samples)
v_upper

951.9014596195834

In [66]:
# Upper limit of the confidence interval
v_mean - stats.norm.ppf(alpha/2)*v_std/np.sqrt(n_samples)

983.1683272487589

In [67]:
primal_solution_df = ps_storage.to_dataframe()
primal_solution_df

x                     y     zx                    zy
i k t                                                          
0 0 0  [1.0]   [96.62592434916624]  [0.0]                 [0.0]
    1  [1.0]   [200.1281459402873]  [1.0]   [96.62592434916624]
    2  [1.0]  [192.65353469111164]  [1.0]   [200.1281459402873]
  1 0  [1.0]   [96.62592434916624]  [1.0]  [192.65353469111164]
    1  [1.0]  [188.84782941332801]  [1.0]   [96.62592434916624]
    2  [1.0]  [192.65353469111164]  [1.0]  [188.84782941332801]

### Backward pass

In [68]:
########################################################################################################################
# Backward pass
########################################################################################################################


binarizer = utils.Binarizer()

sg_method = dualsolver.SubgradientMethod(max_iterations=10)

cut_lower_bounds = [theta_lb]*n_stages
samples = [[0,1], [0,0]]
n_samples = len(samples)
v_opt_k =[]


i = 0
for k in range(n_samples):
    for t in reversed(range(1,n_stages)):
        n_realizations = n_nodes_per_stage[t]
        ds_dict = ds_storage.create_empty_result_dict()
        cc_dict = cc_storage.create_empty_result_dict()
        
        for n in range(n_realizations):

            # TODO Binarization
            bin_vars = []
            bin_multipliers = []
            if t>0:
                float_vars = ps_storage.get_result(i,k,t-1)[y_key]
                x_binary_trial_point = ps_storage.get_result(i,k,t-1)[x_key]
            else:
                #TODO Approximation needed?
                # Might lead to active penalty
                float_vars = np.zeros(n_gens)
                x_binary_trial_point = np.zeros(n_gens)
            
            for j in range(len(float_vars)):
                new_vars, new_multipliers = binarizer.binary_expansion(float_vars[j], upper_bound=pg_max, precision=0.1)
                bin_vars += new_vars
                bin_multipliers.append(new_multipliers) 


            y_binary_trial_point = bin_vars
            y_binary_trial_multipliers = linalg.block_diag(*bin_multipliers)


            uc_bw = ucmodel.BackwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

            uc_bw.add_objective(cost_coeffs)

            uc_bw.add_balance_constraints(sum(p_d[t][n]))

            uc_bw.add_generator_constraints(pg_min, pg_max)

            uc_bw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

            uc_bw.add_startup_shutdown_constraints()

            uc_bw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

            uc_bw.add_relaxation(x_binary_trial_point, y_binary_trial_point)

            uc_bw.add_copy_constraints(y_binary_trial_multipliers)

            uc_bw.add_cut_lower_bound(cut_lower_bounds[t])
            
            if t < n_stages-1:
                cut_coefficients = cc_storage.get_stage_result(t)
                uc_bw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key])

            objective_terms = uc_bw.objective_terms
            relaxed_terms = uc_bw.relaxed_terms
            
            # Solve problem with subgradient method
            #uc_bw.disable_output()
            #uc_bw.model.optimize()
            #uc_bw.model.display()
            sg_method.output_flag = True            
            model, sg_results = sg_method.solve(uc_bw.model, objective_terms, relaxed_terms, 2000)
            print(f"Best lower bound:{sg_results.obj_value}")
            print(f"Best multipliers: {sg_results.multipliers}")
            # if t== 1 and n == 0:
            #     model.setParam("OutputFlag", 1)
            #     model.printAttr("X")
            #     model.display()

            
            # Dual value and multiplier for each realization
            ds_dict[dv_key].append(sg_results.obj_value)
            ds_dict[dm_key].append(sg_results.multipliers)
            
        
        ds_storage.add_result(i, k, t, ds_dict)

        # Calculate and store cut coefficients
        probabilities = prob[t]        
        intercept = np.array(probabilities).dot(np.array(ds_dict[dv_key]))
        gradient = np.array(probabilities).dot(np.array(ds_dict[dm_key]))

        cc_dict[ci_key] = intercept.tolist()
        cc_dict[cg_key] = gradient.tolist()
        cc_dict[bm_key] = y_binary_trial_multipliers

        if t > 0 : cc_storage.add_result(i, k, t-1, cc_dict)
                

Subgradient Method started
New lower bound
Iteration: 0 | Optimal value: 385.307069382221 | Gradient magnitude: 2.686243193201165 | Step size: 0.37226711361465065
Iteration: 1 | Optimal value: 385.3070693822209 | Gradient magnitude: 2.449489742783178 | Step size: 0.4082482904638631
Iteration: 2 | Optimal value: 385.3070693822209 | Gradient magnitude: 0.0
Subgradient Method finished (Tolerance)
Best lower bound:385.307069382221
Best multipliers: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Subgradient Method started
New lower bound
Iteration: 0 | Optimal value: 400.0438944230973 | Gradient magnitude: 2.934147879453782 | Step size: 0.3408144514468572
Iteration: 1 | Optimal value: 400.0438944230972 | Gradient magnitude: 2.23606797749979 | Step size: 0.4472135954999579
New lower bound
Iteration: 2 | Optimal value: 400.04389442309736 | Gradient magnitude: 0.0
Subgradient Method finished (Tolerance)
Best lower bound:400.04389442309736
Best multipliers: [ 0.34081445  0.26601518  0.340

In [69]:
# One dual value and one set of dual multipliers for each realization
ds_storage.to_dataframe()

dual_value  \
i k t                                           
0 0 2  [385.307069382221, 400.04389442309736]   
    1  [770.2007335035918, 792.7613665575102]   
  1 2  [385.307069382221, 400.04389442309736]   
    1  [770.2007335035918, 792.7613665575102]   

                                         dual_multiplier  
i k t                                                     
0 0 2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
    1  [[0.3284045329353418, 0.17133057149172198, 0.3...  
  1 2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
    1  [[0.3284045329353418, 0.17133057149172198, 0.3...

In [70]:
cc_storage.to_dataframe()

intercepts                                          gradients  \
i k t                                                                  
0 0 1  392.675482  [0.1704072257234286, 0.1330075913997228, 0.170...   
    0  781.481050  [0.3296864963876283, 0.25114951566581845, 0.22...   
  1 1  392.675482  [0.1704072257234286, 0.1330075913997228, 0.170...   
    0  781.481050  [0.3296864963876283, 0.25114951566581845, 0.22...   

                                             multipliers  
i k t                                                     
0 0 1  [[0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25....  
    0  [[0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25....  
  1 1  [[0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25....  
    0  [[0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25....

### Lower Bound Update

In [71]:
i = 1
t = 0
n = 0

x_trial_point = [0]*n_gens
y_trial_point = [0]*n_gens

# Create forward model
uc_fw = ucmodel.ForwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

uc_fw.add_objective(cost_coeffs)

uc_fw.add_balance_constraints(sum(p_d[t][n]))

uc_fw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

uc_fw.add_generator_constraints(pg_min, pg_max)

uc_fw.add_startup_shutdown_constraints()

uc_fw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

uc_fw.add_copy_constraints(x_trial_point, y_trial_point)

#TODO Lower bound
uc_fw.add_cut_lower_bound(theta_lb)

if i>0:
    cut_coefficients = cc_storage.get_stage_result(t)
    uc_fw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key])

# Solve problem
#uc_fw.disable_output()
uc_fw.model.optimize()
uc_fw.model.printAttr("X")
    


# Store xtik, ytik, ztik, vtik
x_kt = [x_g.x for x_g in uc_fw.x]
y_kt = [y_g.x for y_g in uc_fw.y]
z_x_kt = [z_g.x for z_g in uc_fw.z_x]
z_y_kt = [z_g.x for z_g in uc_fw.z_y]
s_up_kt = [s_up_g.x for s_up_g in uc_fw.s_up]
s_down_kt = [s_down_g.x for s_down_g in uc_fw.s_down]

# Value of stage t objective function
v_lower = uc_fw.model.getObjective().getValue()

v_lower


974.0735257361082

In [72]:
uc_fw.enable_output()
uc_fw.model.printAttr("X")

Set parameter OutputFlag to value 1

    Variable            X 
-------------------------
         x_1            1 
         y_1      96.6259 
      s_up_1            1 
       theta      780.822 
       ny[0]     0.670314 
      ny[11]     0.329686 
      ny[12]     0.329686 
      ny[14]     0.329686 
       my[0]            1 
       my[1]      0.25115 
       my[2]     0.223656 
       my[3]     0.329686 
       my[4]     0.329686 
       my[5]     0.415484 
       my[6]          0.5 
       my[7]          0.5 
       my[8]     0.414202 
       my[9]     0.329686 
      my[10]          0.5 
   lambda[0]            1 
  lambda[11]            1 
  lambda[12]            1 
  lambda[14]            1 
        w[0]            1 
        w[1]            1 
        w[2]            1 
        w[3]            1 
        w[4]            1 
        w[5]            1 
        w[6]            1 
        w[7]            1 
        w[8]            1 
        w[9]            1 
       w[10]       

In [73]:
# model.setParam("DualReductions", 0)
# model.reset()
# model.optimize()

In [74]:
# model.computeIIS()
# if model.IISMinimal:
#     print("IIS is minimal.")
# else:
#     print("IIS is not minimal.")
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s'%model.constrName)

In [75]:
# model.write("iis"+".ilp")